# Analyze Results

In [ ]:
import os.path
import numpy as np
import pandas as pd

from hit_prediction_code.analytics import get_results_as_dataframe
from hit_prediction_code import analytics

results = get_results_as_dataframe(
    project_name='hit-prediction-ismir2020',
    table_name='hit_prediction',
    filter_git_dirty=True,
    date_filter='> \'2021-02-09 16:15:00\'',
    columns=['id', 'sourcefile', 'outcome'],
)

analytics.add_approach_to_df(results)
analytics.add_cv_epoch_evaluator_outcome_to_df(results)

results.sort_values(by='sourcefile', inplace=True)

In [ ]:
normalize = 'true'

for _, result in results.iterrows():
    mean_result = result['mean']

    normalized_cm = []
    for cm in mean_result.loc['confusion_matrix']:
        if normalize:
            cm = analytics.normalize_confusion_matrix(cm, method=normalize)
        else:
            cm = np.array(cm)
        normalized_cm.append(cm)
    
    display(analytics.scores_from_confusion_matrices(normalized_cm, mean_result.columns))
    plot_shape = (2, 2) 
    if len(normalized_cm) >= plot_shape[0] * plot_shape[1]:
        analytics.plot_epochs_confution_matrix(result['approach'], normalized_cm, mean_result.columns, plot_shape=plot_shape, figsize=(18, 14))